## Vector DBs & Vector Retrievers

### Documents

LangChain implements a Document abstraction, which is intended to represent a unit of
text and associated metadata.

- page_content: a string representing content
- metadata: a dict containing arbitrary metadata. The metadata attribute can capture
  information about the source of the document, its relationship to other documents, and
  other information. Note that an individual Document object often represents a chunk of a
  larger document.

Note: Documents can represent peices of a larger document. Like each document can represent a page

In [5]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    
]

documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [6]:
# Load environoment variables
import os
from dotenv import load_dotenv
load_dotenv()

# Intiailize ChatGroq
from langchain_groq import ChatGroq

# Initialize environnment variables
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

# Initialize the llm
llm = ChatGroq(groq_api_key=groq_api_key, model="Llama3-70B-8192")


In [7]:
# Setup the embeddings to use
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
## Vector Stores
from langchain_chroma import Chroma

# All documents are going to get converted into vectors
# Then it gets stored in the chroma db
vectorstore = Chroma.from_documents(documents, embedding=embeddings)
vectorstore

In [10]:
## Vectorstore Similarity Search

vectorstore.similarity_search("cat", k=2)

[Document(id='dc35d53f-bf72-42ff-9570-7008f864503f', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='3754afc2-1a39-488e-932a-0fa4b823ed79', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [11]:
# Async query

await vectorstore.asimilarity_search("cat", k=2)

[Document(id='dc35d53f-bf72-42ff-9570-7008f864503f', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='3754afc2-1a39-488e-932a-0fa4b823ed79', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [12]:
# Here we get the scores

vectorstore.similarity_search_with_score("cat", k=2)

[(Document(id='dc35d53f-bf72-42ff-9570-7008f864503f', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351058602333069),
 (Document(id='3754afc2-1a39-488e-932a-0fa4b823ed79', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351058602333069)]

## Retrievers

LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be
integrated into LangChain Expression Language chains

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g.
synchronos and asynchronous invoke and batch operations) and are designed to be incorporated
into LCEL chains.

We can create a simple version of this ourselves, withough subclassing retriever. I we choose what
method we wish to use to retrieve deocuments, we can crea runnable easily. Below we will build one
around the similarity_search method

Essentially the code below creates a similarity search lambda function that returns the 
top result for each "document" in the list. In this case that is "cat" and "dog".

In [15]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat", "dog"])

[[Document(id='dc35d53f-bf72-42ff-9570-7008f864503f', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='cfba619b-a861-4678-91bb-dd15ae3edc9f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

Vectorstores implement an as_retriever method that will generate a retriever, specifically
a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes
that identify what methods of the underlying vectorstore to call, and how to parameterize them.
For instance, we can replicate the above with the following:

In [18]:
# Here this is implemented using as_retriever
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 1}
)

retriever.batch(["cat", "dog"])

[[Document(id='dc35d53f-bf72-42ff-9570-7008f864503f', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='cfba619b-a861-4678-91bb-dd15ae3edc9f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

## Integrate it with the chain

In [ ]:
## RAG Chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message ="""
Answer this question using the provided contect only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke("Tell me about dogs?")
print(response)

content='According to the provided context, dogs are great companions, known for their loyalty and friendliness.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 82, 'total_tokens': 102, 'completion_time': 0.040870778, 'prompt_time': 0.015243992, 'queue_time': 0.054140114, 'total_time': 0.05611477}, 'model_name': 'Llama3-70B-8192', 'system_fingerprint': 'fp_de9fa14705', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--3ab424a9-791a-44d6-84f3-240545433590-0' usage_metadata={'input_tokens': 82, 'output_tokens': 20, 'total_tokens': 102}
